# Learning of NN

NNにおける学習→損失関数を最小化するようなパラメータを自動で導き出す（パーセプトロンは手動で設定した）  
通常の機械学習手法では、データの特徴量は人間が設計する必要があるが、NNでは特徴量まで含めデータをそのまま学習する  
これは、扱う対象のデータにかかわらず同じフローで処理ができるという利点につながる（認識対象が犬でも人でも手書き文字でも、処理の流れは変わらない）

## Cost function

NNの性能の悪さを表すのが損失関数で、二乗和誤差や交差エントロピー誤差が主に用いられる  
以下のケースでは二乗和誤差を計算しており、その結果が小さいほど予測値が正解に近いことを示す

In [1]:
import numpy as np

def mean_squared_error(y, t):
    return 0.5 * np.sum((y - t) ** 2)

# 正解ラベルが2で、予測も2
t = np.array([0,0,1,0,0,0,0,0,0,0])
y = np.array([0.1, 0.05, 0.6,0,0.05,0.1,0,0.1,0,0])

print(mean_squared_error(y, t))

# 正解ラベルが2で、予測は6
t = np.array([0,0,1,0,0,0,0,0,0,0])
y = np.array([0.1, 0.05, 0.05, 0, 0.05, 0.65, 0, 0.1, 0, 0])

print(mean_squared_error(y, t))

0.09750000000000003
0.675


以下のケースでは交差エントロピー誤差を計算  
交差エントロピー誤差では、基本的に教師データはone-hotベクトルなので、正解ラベルに対しての予測のみを考慮すればOK

In [2]:
def cross_entopy_error(y, t):
    # log0は-infになって正しく計算できなくなるので、微小な値をyに加える
    delta = 1e-7
    return - np.sum(t * np.log(y + delta))

# 正解ラベルが2で、予測も2
t = np.array([0,0,1,0,0,0,0,0,0,0])
y = np.array([0.1, 0.05, 0.6,0,0.05,0.1,0,0.1,0,0])
print(cross_entopy_error(y, t))

# 正解ラベルが2で、予測は6
t = np.array([0,0,1,0,0,0,0,0,0,0])
y = np.array([0.1, 0.05, 0.05, 0, 0.05, 0.65, 0, 0.1, 0, 0])
print(cross_entopy_error(y, t))

0.510825457099338
2.9957302735559908


### Mini batch learning

学習においては、すべての訓練データについて予測と損失関数を求める必要があるが、莫大な訓練データすべてについて損失関数を計算していくのは時間がかかる  
よって、全訓練データから一部のデータを全体の近似として抽出し（ミニバッチ）、そのバッチごとに学習を行う  
以下は、ミニバッチをMNISTから抽出している

In [3]:
from dlbook.dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
print(x_train.shape)
print(t_train.shape)

train_size = x_train.shape[0]
mini_batch = 10
# np.random.choice(a, b)で、0~aの範囲の中で、b個の数字をランダムに取り出す
batch_mask = np.random.choice(train_size, mini_batch)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]
print(batch_mask)

(60000, 784)
(60000, 10)
[11366 33121  1221  4924 15796  1938 57204 57547 46298 54532]


交差エントロピー誤差を、ミニバッチ学習に対応するように修正する

In [4]:
def cross_entropy_error(y, t):
    # 入力データが1つだけの場合も一般的に計算できるように、2次元に整形する
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    
    mini_batch = y.shape[0]
    return - np.sum(t * np.log(y + 1e-7)) / mini_batch

# mini_batch == 10のケース
y = np.array([[0.1, 0.05, 0.05, 0, 0.05, 0.65, 0, 0.1, 0, 0],
            [0.1, 0.05, 0.05, 0, 0.05, 0.65, 0, 0.1, 0, 0],
            [0.1, 0.05, 0.05, 0, 0.05, 0.65, 0, 0.1, 0, 0],
            [0.1, 0.05, 0.05, 0, 0.05, 0.65, 0, 0.1, 0, 0],
            [0.1, 0.05, 0.05, 0, 0.05, 0.65, 0, 0.1, 0, 0],
            [0.1, 0.05, 0.05, 0, 0.05, 0.65, 0, 0.1, 0, 0],
            [0.1, 0.05, 0.05, 0, 0.05, 0.65, 0, 0.1, 0, 0],
            [0.1, 0.05, 0.05, 0, 0.05, 0.65, 0, 0.1, 0, 0],
            [0.1, 0.05, 0.05, 0, 0.05, 0.65, 0, 0.1, 0, 0],
            [0.1, 0.05, 0.05, 0, 0.05, 0.65, 0, 0.1, 0, 0]])

t = np.array([[0,0,1,0,0,0,0,0,0,0],
             [0,0,0,0,0,0,1,0,0,0],
             [1,0,0,0,0,0,0,0,0,0],
             [0,0,0,0,0,1,0,0,0,0],
             [0,1,0,0,0,0,0,0,0,0],
             [0,0,0,1,0,0,0,0,0,0],
             [0,0,0,0,1,0,0,0,0,0],
             [0,0,0,0,0,0,0,1,0,0],
             [0,0,0,0,0,0,0,0,0,1],
             [0,0,0,0,0,0,0,0,1,0]])

print(cross_entropy_error(y, t))

# y.ndim == 1のケース
y = np.array([0.1, 0.05, 0.05, 0, 0.05, 0.65, 0, 0.1, 0, 0.1])
t = np.array([0,0,0,0,0,1,0,0,0,0])
print(cross_entropy_error(y, t))

7.849552437273667
0.4307827622463123


## Numerical differentiation

機械学習では、損失関数の「勾配」を利用してパラメータの学習を行う  
勾配とは、ある関数について、すべての変数についての偏微分をベクトルとしてまとめたものである  
「変数xの微小な変化に対して、関数f(x)の出力がどのくらい変化するか」を表すのが微分であり、以下の式で定義できる  
$$
  \frac{d}{d x} f(x) = \lim_{h \to 0} \frac{f(x+h)-f(x)}{h} 
$$
（後述する誤差逆伝播法では、解析的に偏微分を求める）  

In [5]:
def numerical_diff(f, x):
    # hは丸め誤差が生じない程度の、0に近い慣習的な値
    h = 1e-4
    # f(x + h)とf(x)の差分は前方差分と呼ばれるが、実装上これは誤差が大きくなる(hを0に無限に近づけられないため)
    # よって、f(x + h)とf(x - h)の差分（中心差分）を用いて数値微分を計算する
    return (f(x + h) - f(x - h)) / (2 * h)

ex.$ f(x) = 0.01x^2+0.1x $ をx=5, x=10でそれぞれ数値微分してみると、解析解0.2, 0.3とほぼ一致する

In [6]:
f1 = (lambda x: 0.01 * x ** 2 + 0.1 * x)
print(numerical_diff(f1, 5))
print(numerical_diff(f1, 10))

0.1999999999990898
0.2999999999986347


## Gradient

ある変数以外の変数を定数とみなして微分を行う手法が偏微分で、$ \frac{\partial f}{\partial x_0}, \frac{\partial f}{\partial x_1} $のように表す（それぞれ、fをx0、x1に着目して偏微分する、の意）  
それらを$ (\frac{\partial f}{\partial x_0}, \frac{\partial f}{\partial x_1}) $ という様に、ベクトルとしてまとめてしまったものを勾配という(勾配の形状は、元のパラメータの形状と同じになる）  
以下のように実装可能

In [7]:
def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x) # xと同じ形状の0配列を生成
    
    # 各々の変数について偏微分を計算
    for idx in range(x.size):
        # 中心差分で数値微分
        tmp = x[idx]
        # idx番目のxについて微小に変化させて関数を数値微分することで、idx番目のxについての偏微分が求まる
        x[idx] = tmp + h
        fx1 = f(x)
        x[idx] = tmp - h
        fx2 = f(x)
        
        grad[idx] = (fx1 - fx2) / (2 * h)
        x[idx] = tmp # x[idx]をもとに戻す
    return grad

ex.$ f(x_0,x_1) = x_0^2 + x_1^2$の勾配を計算

In [8]:
f2 = lambda x: x[0]**2 + x[1]**2
print(numerical_gradient(f2, np.array([3.0, 4.0])))
print(numerical_gradient(f2, np.array([0.0, 2.0])))
print(numerical_gradient(f2, np.array([3.0, 0.0])))

[6. 8.]
[0. 4.]
[6. 0.]


これは、関数f2の各点における$ (x_0, x_1) $の勾配を表し、**各点において関数の値を最も減らす方向を示している**

### Gradient descent

勾配は、ある点においてその関数の値を最も減らす方向を表している  
機械学習における学習とは、損失関数の値が最も小さくなるようなパラメータを導くことである  
つまり、勾配を用いてパラメータを更新し、更新したパラメータを用いて再び損失関数＆勾配を計算するのを繰り返していくことで、損失関数を小さくしていくことができる  
これを勾配降下法という

In [9]:
def grad_descent(f, init_x, lr=0.1, step_num=100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        # 勾配*学習率で減算して、パラメータxを更新する
        x -= lr * grad
        
    return x

init_x = np.array([-3.0, 4.0])
# 勾配降下でf2が最小になるときのパラメータxを探索
print(grad_descent(f2, init_x))

[-6.11110793e-10  8.14814391e-10]


学習率が大きすぎor小さすぎると勾配降下法はうまくいかない(以下参照）

In [10]:
# ほとんどxは更新されていない
print(grad_descent(f2, np.array([-3.0, 4.0]), lr=1e-10))
# xが大きすぎる値に発散してしまう
print(grad_descent(f2, np.array([-3.0, 4.0]), lr=10.0))

[-2.99999994  3.99999992]
[-2.58983747e+13 -1.29524862e+12]


NNにおける学習とは、重みパラメータWについての損失関数の勾配を用いて、最適なパラメータWを導くこと  
簡易的なNNを実装し、実際に勾配を求める

In [11]:
from dlbook.common.functions import softmax, cross_entropy_error
from dlbook.common.gradient import numerical_gradient

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3) # mean=0, sigma=1の正規分布で初期化
    
    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        return loss
net = simpleNet()
print(net.W)

[[ 0.40155232  0.33531113 -1.51458275]
 [-0.06299324  1.00012669 -0.01779266]]


In [13]:
x1 = np.array([0.6, 0.9])
p = net.predict(x1)
print(p, np.argmax(p))
t1 = np.array([1,0,0])

[ 0.18423747  1.1013007  -0.92476304] 1


In [16]:
# 損失関数において、パラメータWの勾配dWを導出
fW = lambda W: net.loss(x1, t1)
# net.Wは参照渡しされており、ミュータブルな要素なので、numerical_gradient内での操作はnet.Wに直接反映される
dW = numerical_gradient(fW, net.W) 
dW

array([[-0.44340935,  0.39175507,  0.05165428],
       [-0.66511403,  0.58763261,  0.07748142]])

この勾配は、各パラメータを$ h $だけ動かしたときの損失関数の動きを表している  
ex. $ w_{11} $を$ h $だけ動かすと、損失関数の値は0.44hだけ減少する  

## Summary of learning process

1. ミニバッチ
    - 訓練データの中からランダムに一部のデータを抽出する  
    - 選ばれた一部のデータ群をミニバッチと呼び、ミニバッチについて損失関数の値を減らすことを目指す
2. 勾配の算出
    - ミニバッチの損失関数を減少させるために、各重みパラメータの勾配を求める（損失関数を重みパラメータについて偏微分する）
    - 勾配は、損失関数の値を最も減らす方向を示す
3. パラメータの更新
    - 重みパラメータを勾配方向に微小に更新する
4. 1~3を繰り返す
  
これらのプロセスは、ミニバッチによって無作為抽出されたデータを用いてパラメータ更新を行っているので、確率的勾配降下法と呼ばれる

## Train and Test

実際に上記のプロセスを一貫して実装し、テストデータによる評価を行う（MNISTを使う）

In [20]:
import numpy as np
from dlbook.common.functions import *
from dlbook.common.gradient import numerical_gradient
from dlbook.dataset.mnist import load_mnist


class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
    def predict(self, x):
        # 画像データの推論
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
    
    def loss(self, x, t):
        # 推論と正解ラベルの誤差を計算
        y = self.predict(x)
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        # 推論の精度を計算
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        # パラメータ群の勾配を数値微分によって計算
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads

数値微分による勾配計算はかなり時間がかかるので、実用上は誤差逆伝播法を使う  
また、このモデルが汎化性能を持っているかを確かめるために、1エポックごとに訓練データとテストデータの認識精度を記録する  
（1エポック=学習において、訓練データをすべて使い切ったときのイテレーション回数。訓練データ/ミニバッチサイズで求められる）

In [24]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []
train_acc_list = []
test_acc_list = []
# 1エポックあたりのイテレーション数
iter_per_epoch = max(train_size / batch_size, 1)

# hyper params
iters_num = 1000
train_size = x_train.shape[0]
batch_size = 10
learning_rate = 0.1

network = TwoLayerNet(784, 10, 10)

for i in range(iters_num):
    # ミニバッチ取得
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 勾配計算
    grad = network.numerical_gradient(x_batch, t_batch)
    
    # パラメータ更新
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    # 学習曲線の記録
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # 1エポックごとに精度を計算
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print('train acc: {}, test acc: {}'.format(train_acc, test_acc))

train acc: 0.10218333333333333, test acc: 0.101


KeyboardInterrupt: 